In [1]:
import pandas as pd
import numpy as np

import requests
import json

In [2]:
df = pd.read_csv('广州商铺数据.csv')

In [3]:
df.head()

,cenggao,floor,function_list,jianzhumianji,jinshen,location,loupandizhi,loupanmingcheng,miankuan,release_date,service_price,shangpu_type,shifoufenge,shihejingying,titile,total_price,url,zhuangxiu,shiyongmianji
0,3.00米,第1层(共1层),"网络,停车位,上水,下水,可明火,排烟,排污,燃气,外摆区",16㎡,4.00米,"海珠,东晓南,东晓南路",海珠东晓南路,东晓南路,4.00米,2019/8/10 16:33:56,10.00元/㎡·月,临街门面,否,"餐饮美食,百货超市",海珠东晓南地铁口 临街茶少爷铺位 收租1.5万/月 个人红本 急售,48万,https://gz.shop.fang.com/shou/3_241314728.html,精装修,NaN
1,5.50米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",27㎡,10.00米,"广州周边,佛山,长华国际中心",狮山镇博爱中路68号,长华国际中心,2.70米,2019/9/3 21:44:17,0.00元/㎡·月,住宅底商,否,百货超市,长华国际 主通道位置 现经营大卡司 12 产权清,35万,https://gz.shop.fang.com/shou/3_241977721.html,精装修,NaN
2,7.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",20㎡,5.00米,"天河,体育中心,正佳广场",天河路208号,正佳广场,4.00米,2019/8/27 19:03:03,10.00元/㎡·月,购物中心/百货,是,餐饮美食,急售！新收 正佳广场 贡茶在租18000/月 带产权出售 即买即收租,42万,https://gz.shop.fang.com/shou/3_241783707.html,精装修,NaN
3,7.00米,第1层(共1层),"网络,暖气,停车位,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",25㎡,8.00米,"广州周边,佛山,世博广场",佛山市禅城区汾江南路38号,世博广场,6.00米,2019/9/4 14:21:41,5.00元/㎡·月,住宅底商,是,百货超市,世博广场一楼铺位 魁奇路地铁口旁 月收1万7 带租约急售 产权清晰,40万,https://gz.shop.fang.com/shou/3_241992963.html,简装修,NaN
4,3.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯",20㎡,4.00米,"越秀,解放北,白马服装市场",站南路16号,白马服装市场,5.00米,2019/8/30 17:09:57,0.00元/㎡·月,商业街商铺,否,百货超市,广州白马服装城临街 现营一点点奶茶 月租18200,17.6万,https://gz.shop.fang.com/shou/3_241863803.html,简装修,NaN


In [5]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [6]:
city = '广州'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            location = lst_lploc[i]
            url = url_pre.format(My_key, location, city)
            res = requests.get(url).json()
            if res['status'] == '0':
                dict_location[lst_lpname[i]] = np.nan
                counter +=1
            else:
                try: 
                    dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                    if counter % 250 == 0:
                        print('one batch(250) processed')
                except:
                    dict_location[lst_lpname[i]] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed


In [7]:
dict_location

{'(智慧新城) 3地铁上盖 出售': '113.266530,23.132191',
 '5号停机坪': '113.264940,23.183470',
 'COCO': '113.350993,23.030480',
 'COCO都可': '113.233095,23.093764',
 'FEI定制': '113.247255,23.140167',
 'coco': '113.236075,23.068140',
 'gogo新天地': '113.359639,22.943053',
 '一德路': '113.260670,23.113039',
 '一点点': '113.265588,23.124146',
 '一点点 时尚天河店': nan,
 '一点点时尚天河店': '113.510599,23.175580',
 '一起吃小龙虾': '113.281061,23.091912',
 '万丰路': '113.370765,22.936490',
 '万博中心': '113.342811,23.007947',
 '万国广场': '113.272883,23.101389',
 '万寿路': '113.281441,23.102073',
 '万盛汇': '113.312842,23.140658',
 '万福路': '113.272735,23.119905',
 '万科东荟城': '113.503431,23.158912',
 '万科云城米酷': '113.408242,23.166310',
 '万科里': '113.279292,23.082544',
 '万科金域缇香': '113.250380,23.118145',
 '万科金色梦想-北门': '113.529519,23.163331',
 '万菱广场': '113.262466,23.114545',
 '万菱汇': '113.330170,23.132829',
 '万达广场': '113.349453,23.007481',
 '万达广场(广州增城店)': '113.816549,23.275556',
 '万达广场(白云店)': '113.266326,23.173347',
 '万通国际': '113.250561,23.147326',
 '三元里': '113.266859

In [8]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,19] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [9]:
df['city'] = city

In [10]:
df.head()

,cenggao,floor,function_list,jianzhumianji,jinshen,location,loupandizhi,loupanmingcheng,miankuan,release_date,...,shangpu_type,shifoufenge,shihejingying,titile,total_price,url,zhuangxiu,shiyongmianji,location_coords,city
0,3.00米,第1层(共1层),"网络,停车位,上水,下水,可明火,排烟,排污,燃气,外摆区",16㎡,4.00米,"海珠,东晓南,东晓南路",海珠东晓南路,东晓南路,4.00米,2019/8/10 16:33:56,...,临街门面,否,"餐饮美食,百货超市",海珠东晓南地铁口 临街茶少爷铺位 收租1.5万/月 个人红本 急售,48万,https://gz.shop.fang.com/shou/3_241314728.html,精装修,NaN,"113.292090,23.073100",广州
1,5.50米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",27㎡,10.00米,"广州周边,佛山,长华国际中心",狮山镇博爱中路68号,长华国际中心,2.70米,2019/9/3 21:44:17,...,住宅底商,否,百货超市,长华国际 主通道位置 现经营大卡司 12 产权清,35万,https://gz.shop.fang.com/shou/3_241977721.html,精装修,NaN,"113.282472,23.125633",广州
2,7.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",20㎡,5.00米,"天河,体育中心,正佳广场",天河路208号,正佳广场,4.00米,2019/8/27 19:03:03,...,购物中心/百货,是,餐饮美食,急售！新收 正佳广场 贡茶在租18000/月 带产权出售 即买即收租,42万,https://gz.shop.fang.com/shou/3_241783707.html,精装修,NaN,"113.327990,23.132189",广州
3,7.00米,第1层(共1层),"网络,暖气,停车位,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",25㎡,8.00米,"广州周边,佛山,世博广场",佛山市禅城区汾江南路38号,世博广场,6.00米,2019/9/4 14:21:41,...,住宅底商,是,百货超市,世博广场一楼铺位 魁奇路地铁口旁 月收1万7 带租约急售 产权清晰,40万,https://gz.shop.fang.com/shou/3_241992963.html,简装修,NaN,NaN,广州
4,3.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯",20㎡,4.00米,"越秀,解放北,白马服装市场",站南路16号,白马服装市场,5.00米,2019/8/30 17:09:57,...,商业街商铺,否,百货超市,广州白马服装城临街 现营一点点奶茶 月租18200,17.6万,https://gz.shop.fang.com/shou/3_241863803.html,简装修,NaN,"113.253709,23.146566",广州


In [11]:
df.to_csv('广州商铺地址数据.csv', encoding = 'utf-8-sig',index = False)